# Advent of Code 2021

## Day 0 : Imports and Utility Functions

In [90]:
import copy

In [91]:
def file_to_list(filename, sep="\n", maxsplit=-1) -> list:
    """
    Read an input file and split it using sep as the delimiter.
    """
    with open(filename) as f:
        return f.read().rstrip().split(sep, maxsplit=maxsplit)

## Day 1: Sonar Sweep

### Part 1

Given a sonar report, count the number of times a depth measurement increases from the previous measurement. For example, in the following report, there are 7 measurements that are larger than the previous measurement.

In [92]:
test_report = [199, 200, 208, 210, 200, 207, 240, 269, 260, 263]

In [93]:
def day1_part1(report: list[int]):
    n = 0
    for i in reversed(range(1, len(report))):
        if report[i] > report[i-1]:
            n += 1
    return n

day1_part1(test_report)

7

In [94]:
final_report = [int(value) for value in file_to_list("input.txt")]
day1_part1(final_report)

1215

### Part 2

Consider sums of a three-measurement sliding window. How many sums are larger than the previous sum?

In [95]:
# Number of sliding windows
len(test_report) - 2

8

In [96]:
def day1_part2(report: list[int]):
    n = 0
    for i in reversed(range(3, len(report))):
        if sum(report[i-2:i+1]) > sum(report[i-3:i]):
            n += 1
    return n

day1_part2(test_report)

5

In [97]:
day1_part2(final_report)

1150

## Day 2: Dive!

### Part 1

The submarine has a planned course consisting of a list of commands such as `forward 1`, `down 2`, or `up 3`. The horizontal position and depth both start at `0`. Calculate the horizontal position and depth you would have after following the planned course. What do you get if you multiply your final horizontal position by your final depth?

In [98]:
test_course = ["forward 5", "down 5", "forward 8", "up 3", "down 8", "forward 2"]

In [99]:
def day2_part1(course: list[str]):
    # x: horizontal position, y: depth
    x = y = 0
    for cmd in course:
        d = int(cmd[-1])
        if cmd[0] == "f":
            x += d
        elif cmd[0] == "d":
            y += d
        else:
            y -= d
    return x * y

day2_part1(test_course)

150

In [100]:
final_course = file_to_list("input2.txt")
day2_part1(final_course)

2150351

### Part 2

New interpretation of the commands:

- `down X` increases your aim by X units.
- `up X` decreases your aim by X units.
- `forward X` does two things:
    - It increases your horizontal position by X units.
    - It increases your depth by your aim multiplied by X.

Using this new interpretation of the commands, calculate the horizontal position and depth you would have after following the planned course. What do you get if you multiply your final horizontal position by your final depth?

In [101]:
def day2_part2(course: list[str]):
    # x: horizontal position, y: depth
    x = y = aim = 0
    for cmd in course:
        d = int(cmd[-1])
        if cmd[0] == "f":
            x += d
            y += aim * d
        elif cmd[0] == "d":
            aim += d
        else:
            aim -= d
    return x * y

day2_part2(test_course)

900

In [102]:
day2_part2(final_course)

1842742223

## Day 3: Binary Diagnostic

### Part 1

The puzzle input (a diagnostic report) consists of a list of binary numbers. You need to use the binary numbers in the diagnostic report to generate two new binary numbers (called the `gamma rate` and the `epsilon rate`). Each bit in the `gamma rate` can be determined by finding the most common bit in the corresponding position of all numbers in the diagnostic report. The `epsilon rate` is calculated in a similar way; rather than use the most common bit, the least common bit from each position is used.

Use the binary numbers in your diagnostic report to calculate the `gamma rate` and `epsilon rate`, then multiply them together. What is the power consumption of the submarine? (Be sure to represent your answer in decimal, not binary.)

In [103]:
test_report = ["00100", "11110", "10110", "10111", "10101", "01111", "00111", "11100", "10000", "11001", "00010", "01010"]

In [104]:
def day3_part1(report: list[str]):
    gamma = ""
    for i in range(len(report[0])):
        n0 = 0
        for j in range(len(report)):
            if report[j][i] == "0":
                n0 += 1
        if n0 >= len(report) // 2:
            gamma += "0"
        else:
            gamma += "1"
    gamma = int(gamma, 2)
    epsilon = 2 ** len(report[0]) - 1 - gamma
    return gamma * epsilon

day3_part1(test_report)

198

In [105]:
final_report = file_to_list("input3.txt")
day3_part1(final_report)

841526

### Part 2

Next, consider the `oxygen generator rating` and the `CO2 scrubber rating`. To find `oxygen generator rating`, determine the most common value (0 or 1) in the current bit position, and keep only numbers with that bit in that position. If 0 and 1 are equally common, keep values with a 1 in the position being considered. To find `CO2 scrubber rating`, determine the least common value (0 or 1) in the current bit position, and keep only numbers with that bit in that position. If 0 and 1 are equally common, keep values with a 0 in the position being considered.

Use the binary numbers in your diagnostic report to calculate the `oxygen generator rating` and `CO2 scrubber rating`, then multiply them together. What is the life support rating of the submarine? (Be sure to represent your answer in decimal, not binary.)

In [106]:
def day3_part2(report: list[str]):
    oxygen = c02 = report
    for i in range(len(oxygen[0])):
        if len(oxygen) > 1:
            n0 = 0
            for j in range(len(oxygen)):
                if oxygen[j][i] == "0":
                    n0 += 1
            most = "1"
            if n0 > len(oxygen) // 2:
                most = "0"
            oxygen = [o for o in oxygen if o[i] == most]
    for i in range(len(c02[0])):
        if len(c02) > 1:
            n0 = 0
            for j in range(len(c02)):
                if c02[j][i] == "0":
                    n0 += 1
            least = "1"
            if n0 <= len(c02) // 2:
                least = "0"
            c02 = [c for c in c02 if c[i] == least]
    return int(oxygen[0], 2) * int(c02[0], 2)

day3_part2(test_report)

230

In [107]:
day3_part2(final_report)

4790390

## Day 4: Giant Squid

### Part 1

Play bingo with the giant squid that has attached itself to the submarine.

Given a random sequence of numbers and random set of boards, find the winning board. If all numbers in any row or any column of a board are marked, that board wins. Find the sum of all unmarked numbers on the winning board and multiply that sum by the number that was just called when the board won.

In [108]:
test_draw_numbers = [7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1]
test_boards = [[[22, 13, 17, 11, 0],
  [8, 2, 23, 4, 24],
  [21, 9, 14, 16, 7],
  [6, 10, 3, 18, 5],
  [1, 12, 20, 15, 19]],
 [[3, 15, 0, 2, 22],
  [9, 18, 13, 17, 5],
  [19, 8, 7, 25, 23],
  [20, 11, 10, 24, 4],
  [14, 21, 16, 12, 6]],
 [[14, 21, 17, 24, 4],
  [10, 16, 15, 9, 19],
  [18, 8, 23, 26, 20],
  [22, 11, 13, 6, 5],
  [2, 0, 12, 3, 7]]]

In [109]:
def day4_part1(numbers: list[str], boards: list[list]):
    winner = None
    for number in numbers:
        for board in boards:
            # Mark the new number
            for i in range(5):
                h = v = 0
                for j in range(5):
                    if number == board[i][j]:
                        board[i][j] = -1
                    if board[i][j] == -1:
                        h += 1
                    if board[j][i] == -1:
                        v += 1
                # Check if the board has won
                if h == 5 or v == 5:
                    winner = board
                    break
            if winner:
                score = sum(sum(x for x in row if x != -1) for row in board)
                print(score * number)
                return

day4_part1(copy.deepcopy(test_draw_numbers), copy.deepcopy(test_boards))

4512


In [110]:
input4 = file_to_list("input4.txt", "\n\n", 1)
final_draw_numbers = [int(n) for n in input4[0].split(",")]
final_boards = [x.split("\n") for x in input4[1].split("\n\n")]
final_boards = [[[int(z) for z in y.split()] for y in x] for x in final_boards]

In [111]:
day4_part1(copy.deepcopy(final_draw_numbers), copy.deepcopy(final_boards))

27027


### Part 2

Let's try a different strategy: let the giant squid win.

Figure out which board will win last. Once it wins, what would its final score be?

In [112]:
def day4_part2(numbers: list[str], boards: list[list]):
    for number in numbers:
        for b in reversed(range(len(boards))):
            won = False
            # Mark the new number
            for i in range(5):
                h = v = 0
                for j in range(5):
                    if number == boards[b][i][j]:
                        boards[b][i][j] = -1
                    if boards[b][i][j] == -1:
                        h += 1
                    if boards[b][j][i] == -1:
                        v += 1
                # Check if the board has won
                if h == 5 or v == 5:
                    won = True            
            if won == True:
                if len(boards) == 1:
                    score = sum(sum(x for x in row if x != -1) for row in boards[b])
                    print(score * number)
                boards.remove(boards[b])

day4_part2(copy.deepcopy(test_draw_numbers), copy.deepcopy(test_boards))

1924


In [113]:
day4_part2(final_draw_numbers, final_boards)

36975
